# Import required libraries

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

#Device configuration
device=torch.device('cpu')
if torch.cuda.is_available():
    device=torch.device('cuda')

# Hyper-parameters

In [3]:
num_epochs = 20
batch_size = 64
learning_rate = 0.001

# Preprocessing

In [4]:
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()]
)

# Load dataset

In [5]:
# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='/data',train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.CIFAR10(root='/data', train=False, transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size,  shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size,  shuffle=False)

Files already downloaded and verified


# Model Architecture

In [6]:
#3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

#Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

#ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Optimizer and Loss function

In [7]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Updating learning rate

In [8]:
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Train the model

In [9]:
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        #Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}".format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    #Decay learning rate
    if (epoch+1) % 5 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch [1/20], Step [10/782] Loss: 1.9326
Epoch [1/20], Step [20/782] Loss: 1.6759
Epoch [1/20], Step [30/782] Loss: 1.7055
Epoch [1/20], Step [40/782] Loss: 1.7789
Epoch [1/20], Step [50/782] Loss: 1.7832
Epoch [1/20], Step [60/782] Loss: 1.8805
Epoch [1/20], Step [70/782] Loss: 1.4547
Epoch [1/20], Step [80/782] Loss: 1.6089
Epoch [1/20], Step [90/782] Loss: 1.5947
Epoch [1/20], Step [100/782] Loss: 1.4633
Epoch [1/20], Step [110/782] Loss: 1.6182
Epoch [1/20], Step [120/782] Loss: 1.6323
Epoch [1/20], Step [130/782] Loss: 1.4564
Epoch [1/20], Step [140/782] Loss: 1.5729
Epoch [1/20], Step [150/782] Loss: 1.4476
Epoch [1/20], Step [160/782] Loss: 1.4388
Epoch [1/20], Step [170/782] Loss: 1.5623
Epoch [1/20], Step [180/782] Loss: 1.4105
Epoch [1/20], Step [190/782] Loss: 1.4043
Epoch [1/20], Step [200/782] Loss: 1.4312
Epoch [1/20], Step [210/782] Loss: 1.4264
Epoch [1/20], Step [220/782] Loss: 1.3969
Epoch [1/20], Step [230/782] Loss: 1.4499
Epoch [1/20], Step [240/782] Loss: 1.3748
E

Epoch [3/20], Step [410/782] Loss: 0.6476
Epoch [3/20], Step [420/782] Loss: 0.8410
Epoch [3/20], Step [430/782] Loss: 0.9075
Epoch [3/20], Step [440/782] Loss: 1.0006
Epoch [3/20], Step [450/782] Loss: 0.6643
Epoch [3/20], Step [460/782] Loss: 0.8532
Epoch [3/20], Step [470/782] Loss: 0.7784
Epoch [3/20], Step [480/782] Loss: 0.8693
Epoch [3/20], Step [490/782] Loss: 0.7207
Epoch [3/20], Step [500/782] Loss: 0.7146
Epoch [3/20], Step [510/782] Loss: 0.5763
Epoch [3/20], Step [520/782] Loss: 0.6099
Epoch [3/20], Step [530/782] Loss: 0.8990
Epoch [3/20], Step [540/782] Loss: 0.8000
Epoch [3/20], Step [550/782] Loss: 0.7242
Epoch [3/20], Step [560/782] Loss: 0.6517
Epoch [3/20], Step [570/782] Loss: 1.0942
Epoch [3/20], Step [580/782] Loss: 0.5370
Epoch [3/20], Step [590/782] Loss: 0.6165
Epoch [3/20], Step [600/782] Loss: 0.4876
Epoch [3/20], Step [610/782] Loss: 0.9043
Epoch [3/20], Step [620/782] Loss: 0.8868
Epoch [3/20], Step [630/782] Loss: 0.8295
Epoch [3/20], Step [640/782] Loss:

Epoch [6/20], Step [30/782] Loss: 0.3544
Epoch [6/20], Step [40/782] Loss: 0.3273
Epoch [6/20], Step [50/782] Loss: 0.3372
Epoch [6/20], Step [60/782] Loss: 0.5336
Epoch [6/20], Step [70/782] Loss: 0.2764
Epoch [6/20], Step [80/782] Loss: 0.3467
Epoch [6/20], Step [90/782] Loss: 0.6507
Epoch [6/20], Step [100/782] Loss: 0.3961
Epoch [6/20], Step [110/782] Loss: 0.2682
Epoch [6/20], Step [120/782] Loss: 0.4271
Epoch [6/20], Step [130/782] Loss: 0.3479
Epoch [6/20], Step [140/782] Loss: 0.2772
Epoch [6/20], Step [150/782] Loss: 0.3200
Epoch [6/20], Step [160/782] Loss: 0.5803
Epoch [6/20], Step [170/782] Loss: 0.2621
Epoch [6/20], Step [180/782] Loss: 0.4831
Epoch [6/20], Step [190/782] Loss: 0.4795
Epoch [6/20], Step [200/782] Loss: 0.2652
Epoch [6/20], Step [210/782] Loss: 0.4707
Epoch [6/20], Step [220/782] Loss: 0.3050
Epoch [6/20], Step [230/782] Loss: 0.4491
Epoch [6/20], Step [240/782] Loss: 0.5487
Epoch [6/20], Step [250/782] Loss: 0.3064
Epoch [6/20], Step [260/782] Loss: 0.4535

Epoch [8/20], Step [440/782] Loss: 0.2656
Epoch [8/20], Step [450/782] Loss: 0.4006
Epoch [8/20], Step [460/782] Loss: 0.3696
Epoch [8/20], Step [470/782] Loss: 0.2686
Epoch [8/20], Step [480/782] Loss: 0.3481
Epoch [8/20], Step [490/782] Loss: 0.3598
Epoch [8/20], Step [500/782] Loss: 0.2294
Epoch [8/20], Step [510/782] Loss: 0.4387
Epoch [8/20], Step [520/782] Loss: 0.4668
Epoch [8/20], Step [530/782] Loss: 0.2777
Epoch [8/20], Step [540/782] Loss: 0.4281
Epoch [8/20], Step [550/782] Loss: 0.4030
Epoch [8/20], Step [560/782] Loss: 0.5613
Epoch [8/20], Step [570/782] Loss: 0.3001
Epoch [8/20], Step [580/782] Loss: 0.3374
Epoch [8/20], Step [590/782] Loss: 0.3815
Epoch [8/20], Step [600/782] Loss: 0.3305
Epoch [8/20], Step [610/782] Loss: 0.4842
Epoch [8/20], Step [620/782] Loss: 0.2402
Epoch [8/20], Step [630/782] Loss: 0.3581
Epoch [8/20], Step [640/782] Loss: 0.3564
Epoch [8/20], Step [650/782] Loss: 0.4405
Epoch [8/20], Step [660/782] Loss: 0.5470
Epoch [8/20], Step [670/782] Loss:

Epoch [11/20], Step [40/782] Loss: 0.1383
Epoch [11/20], Step [50/782] Loss: 0.1371
Epoch [11/20], Step [60/782] Loss: 0.1662
Epoch [11/20], Step [70/782] Loss: 0.2341
Epoch [11/20], Step [80/782] Loss: 0.2208
Epoch [11/20], Step [90/782] Loss: 0.1640
Epoch [11/20], Step [100/782] Loss: 0.1420
Epoch [11/20], Step [110/782] Loss: 0.1657
Epoch [11/20], Step [120/782] Loss: 0.1521
Epoch [11/20], Step [130/782] Loss: 0.0914
Epoch [11/20], Step [140/782] Loss: 0.1338
Epoch [11/20], Step [150/782] Loss: 0.2519
Epoch [11/20], Step [160/782] Loss: 0.1644
Epoch [11/20], Step [170/782] Loss: 0.1044
Epoch [11/20], Step [180/782] Loss: 0.2876
Epoch [11/20], Step [190/782] Loss: 0.1687
Epoch [11/20], Step [200/782] Loss: 0.1228
Epoch [11/20], Step [210/782] Loss: 0.1394
Epoch [11/20], Step [220/782] Loss: 0.1528
Epoch [11/20], Step [230/782] Loss: 0.1358
Epoch [11/20], Step [240/782] Loss: 0.1463
Epoch [11/20], Step [250/782] Loss: 0.2027
Epoch [11/20], Step [260/782] Loss: 0.2049
Epoch [11/20], St

Epoch [13/20], Step [400/782] Loss: 0.2012
Epoch [13/20], Step [410/782] Loss: 0.1842
Epoch [13/20], Step [420/782] Loss: 0.1078
Epoch [13/20], Step [430/782] Loss: 0.1225
Epoch [13/20], Step [440/782] Loss: 0.2029
Epoch [13/20], Step [450/782] Loss: 0.1907
Epoch [13/20], Step [460/782] Loss: 0.2014
Epoch [13/20], Step [470/782] Loss: 0.1278
Epoch [13/20], Step [480/782] Loss: 0.2379
Epoch [13/20], Step [490/782] Loss: 0.0799
Epoch [13/20], Step [500/782] Loss: 0.1431
Epoch [13/20], Step [510/782] Loss: 0.1971
Epoch [13/20], Step [520/782] Loss: 0.1818
Epoch [13/20], Step [530/782] Loss: 0.0825
Epoch [13/20], Step [540/782] Loss: 0.3670
Epoch [13/20], Step [550/782] Loss: 0.2714
Epoch [13/20], Step [560/782] Loss: 0.2052
Epoch [13/20], Step [570/782] Loss: 0.1399
Epoch [13/20], Step [580/782] Loss: 0.1338
Epoch [13/20], Step [590/782] Loss: 0.1459
Epoch [13/20], Step [600/782] Loss: 0.1283
Epoch [13/20], Step [610/782] Loss: 0.1171
Epoch [13/20], Step [620/782] Loss: 0.1375
Epoch [13/2

Epoch [15/20], Step [750/782] Loss: 0.1273
Epoch [15/20], Step [760/782] Loss: 0.1108
Epoch [15/20], Step [770/782] Loss: 0.1316
Epoch [15/20], Step [780/782] Loss: 0.1611
Epoch [16/20], Step [10/782] Loss: 0.2259
Epoch [16/20], Step [20/782] Loss: 0.1327
Epoch [16/20], Step [30/782] Loss: 0.1226
Epoch [16/20], Step [40/782] Loss: 0.1133
Epoch [16/20], Step [50/782] Loss: 0.1015
Epoch [16/20], Step [60/782] Loss: 0.1781
Epoch [16/20], Step [70/782] Loss: 0.1093
Epoch [16/20], Step [80/782] Loss: 0.0701
Epoch [16/20], Step [90/782] Loss: 0.1334
Epoch [16/20], Step [100/782] Loss: 0.1593
Epoch [16/20], Step [110/782] Loss: 0.0935
Epoch [16/20], Step [120/782] Loss: 0.1425
Epoch [16/20], Step [130/782] Loss: 0.0784
Epoch [16/20], Step [140/782] Loss: 0.0613
Epoch [16/20], Step [150/782] Loss: 0.1722
Epoch [16/20], Step [160/782] Loss: 0.1587
Epoch [16/20], Step [170/782] Loss: 0.1304
Epoch [16/20], Step [180/782] Loss: 0.1251
Epoch [16/20], Step [190/782] Loss: 0.0971
Epoch [16/20], Step 

Epoch [18/20], Step [330/782] Loss: 0.0862
Epoch [18/20], Step [340/782] Loss: 0.1261
Epoch [18/20], Step [350/782] Loss: 0.0984
Epoch [18/20], Step [360/782] Loss: 0.1141
Epoch [18/20], Step [370/782] Loss: 0.0882
Epoch [18/20], Step [380/782] Loss: 0.1320
Epoch [18/20], Step [390/782] Loss: 0.0310
Epoch [18/20], Step [400/782] Loss: 0.1373
Epoch [18/20], Step [410/782] Loss: 0.1528
Epoch [18/20], Step [420/782] Loss: 0.1178
Epoch [18/20], Step [430/782] Loss: 0.1244
Epoch [18/20], Step [440/782] Loss: 0.1110
Epoch [18/20], Step [450/782] Loss: 0.1437
Epoch [18/20], Step [460/782] Loss: 0.0907
Epoch [18/20], Step [470/782] Loss: 0.1615
Epoch [18/20], Step [480/782] Loss: 0.0947
Epoch [18/20], Step [490/782] Loss: 0.1853
Epoch [18/20], Step [500/782] Loss: 0.1222
Epoch [18/20], Step [510/782] Loss: 0.0460
Epoch [18/20], Step [520/782] Loss: 0.0750
Epoch [18/20], Step [530/782] Loss: 0.0940
Epoch [18/20], Step [540/782] Loss: 0.0775
Epoch [18/20], Step [550/782] Loss: 0.1312
Epoch [18/2

Epoch [20/20], Step [680/782] Loss: 0.0520
Epoch [20/20], Step [690/782] Loss: 0.0874
Epoch [20/20], Step [700/782] Loss: 0.1540
Epoch [20/20], Step [710/782] Loss: 0.0744
Epoch [20/20], Step [720/782] Loss: 0.0898
Epoch [20/20], Step [730/782] Loss: 0.0897
Epoch [20/20], Step [740/782] Loss: 0.0699
Epoch [20/20], Step [750/782] Loss: 0.1382
Epoch [20/20], Step [760/782] Loss: 0.1284
Epoch [20/20], Step [770/782] Loss: 0.1203
Epoch [20/20], Step [780/782] Loss: 0.0973


# Test the model

In [10]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 81.77 %
